# word2vec + XGBoost Classifier (BPE: 2000개)
- word2vec: opcode BPE token을 임베딩
- XGBoost Classifier: 임베딩값을 분류
- accuracy: 0.9940119760479041
- dataset1(DikeDataset): https://github.com/iosifache/DikeDataset
- dataset2(GnuWin Packages): http://gnuwin32.sourceforge.net/packages.html
- BPE Token을 만드는 알고리즘은 utils 디렉토리 내부의 BPE Token generator.ipynb를 확인하시면 됩니다.

# 필요 pacakge 설치

In [1]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 666 kB/s 
  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=9cb415044c3b5fba43e852d4f0815f30172b54595c70f2c40c65409d0b2e634c
  Stored in directory: /root/.cache/pip/wheels/50/be/9b/d19a6151b9c6e303f823faedf03193d7e90dac4009a0bdafeb
Successfully built pefile
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 14.4 MB/s 


# utils.py 파일, sample 파일이 있는 폴더로 위치 변경

In [5]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 package Import

In [6]:
from utils import *
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import time

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Data load

## Vocab Load
- BPE Token 2000개짜리 사전 로드
- 전체 opcode 리스트 로드
- vocab에 전체 opcode 리스트 추가

In [7]:
with open('2000vocab.p', 'rb') as file:
    vocab = pickle.load(file)

filePath = 'opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)

opcodes2 = []
for i in range(len(opcodes)):
    opcodes2.append([opcodes[i]])

vocab = vocab + opcodes2

print(vocab[:20])

[['add', 'add'], ['mov', 'mov'], ['add', 'add', 'add', 'add'], ['push', 'push'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['push', 'call'], ['push', 'mov'], ['dec', 'mov'], ['mov', 'mov', 'mov', 'mov'], ['pop', 'pop'], ['mov', 'cmp'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['test', 'je'], ['mov', 'mov', 'mov'], ['mov', 'jmp'], ['cmp', 'je'], ['mov', 'add'], ['nop', 'nop'], ['pop', 'ret'], ['push', 'push', 'call']]


# Trained word2vec model load

In [8]:
modelPath = 'BPEw2v2000'
word2vecModel = KeyedVectors.load_word2vec_format(modelPath)

## word2vec model test

In [9]:
model_result = word2vecModel.most_similar("addadd")
print(model_result)

[('addaddadd', 0.4414026737213135), ('addaddaddadd', 0.3697534203529358), ('decaddaddadd', 0.3347220718860626), ('pushaddaddaddadd', 0.3212829530239105), ('addaddaddaddadd', 0.3178083300590515), ('adcaddadd', 0.31767505407333374), ('pushaddadd', 0.2995673418045044), ('jnpaddadd', 0.2970563769340515), ('pushaddaddadd', 0.29624754190444946), ('xchgaddadd', 0.2945404648780823)]


# word2vec 임베딩 값을 위한 함수
- word2vec 임베딩 값의 평균을 임베딩 값으로 사용

In [10]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(word2vecModel.wv[i])
        except KeyError as e:
            pass
    try:
        return np.array(np.mean(vector, axis=0), dtype = np.float64)
    except IndexError as e:
        pass

# 모델 로드

In [11]:
# 파일명
filename = 'XGB_BPE2000_Word2vecClassifier.model'

# 모델 불러오기
clf = pickle.load(open(filename, 'rb'))

# Sample file Test
- Test를 위해 utils 폴더 내부의 utils.py 파일을 활용한다.

## MalwareDetectionFunction을 통한 Malware 탐지
- 양성 샘플 파일이 양성으로 결과가 나오는지 확인
- return 1: 악성코드, return 0: 양성코드

In [12]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunction(model, fileName):
    try:
        #parse exe file
        exe = pefile.PE(fileName)
        #call the function we created earlier
        sampleOpcodeList = ExtractPefileOpcodes(opcodes, exe)
    except:
        print('Error File')
    
    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    inputData = get_sentence_mean_vector(sampleTokenList)
    inputData = [inputData]

    return model.predict(inputData)[0]

- 결과가 0으로 양성파일을 탐지한 것을 확인할 수 있다.

In [13]:
fileName = '3f3fe9ecad7f30fc80cdfb678d7ca27a30d0575a73818746e98be9170d3be348.exe'

MalwareDetectionFunction(clf, fileName)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


0

# opcode Sequence List pickle을 입력으로 사용하는 탐지 함수
- 악성코드 파일을 직접 Google Drive 및 GitHub에 올릴 수 없기 때문에 pickle로 먼저 opcodeSequence를 추출하여 해당 파일을 바탕으로 악성코드를 탐지하는 함수를 제작하였다.

In [14]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunctionUsingPickle(model, pickleName):
    with open(pickleName, 'rb') as lf:
        sampleOpcodeList = pickle.load(lf)

    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    inputData = get_sentence_mean_vector(sampleTokenList)
    inputData = [inputData]

    return model.predict(inputData)[0]

- 악성코드 파일이기 때문에 모델이 1로 분류하는 것을 확인할 수 있다.

In [15]:
pickleName = 'MalwareSample.p'

MalwareDetectionFunctionUsingPickle(clf, pickleName)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


1